This code produces a polygon from the DWD 1x1km grid.

In [27]:
from netCDF4 import Dataset, MFDataset
import numpy as np
from datetime import datetime, timedelta
from cftime import num2date, date2num, date2index
import geopandas as gpd
from shapely.geometry import Polygon
from climate_data.lat_lon_mask import lat_lon_mask as mask

In [28]:
nc = Dataset('climate_netcdf/zalf_tasmin_amber_2021_v1-0.nc', 'r')

In [29]:
lats = np.array(mask['lat'])
lons = np.array(mask['lon'])

In [30]:
lat_diffs = np.abs(np.diff(lats))
lon_diffs = np.abs(np.diff(lons))

In [31]:
lat_cell_size = np.mean(lat_diffs)
lon_cell_size = np.mean(lon_diffs)

In [32]:
polygons = []
data=[]

In [33]:
for i in range(len(lats)):
    for j in range(len(lons)):
        if mask['mask'][i][j]:
            lat1 = lats[i] + lat_cell_size / 2
            lat2 = lats[i] - lat_cell_size / 2
            lon1 = lons[j] - lon_cell_size / 2
            lon2 = lons[j] + lon_cell_size / 2
            polygons.append(Polygon([(lon1, lat1), (lon2, lat1), (lon2, lat2), (lon1, lat2)]))

            data.append({
            'lat': lats[i],
            'lon': lons[j],
            'lat_idx': i,
            'lon_idx': j
        })
        else:
            continue

In [34]:
gdf = gpd.GeoDataFrame(data, geometry=polygons)
gdf.crs = 'EPSG:4326'  # Set the coordinate reference system to WGS84

# Save to GeoJSON
gdf.to_file('raster_polygons.geojson', driver='GeoJSON')

# Optionally, save to Shapefile
gdf.to_file('raster_polygons.shp')